In [9]:
import os
from matplotlib import pyplot as plt
from matplotlib.image import imread
from PIL import Image
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import RMSprop

In [10]:
base_dir = 'dogs_breed'    
valid_dir = 'valid'
train_dir = 'train'
train_path = os.path.join(base_dir, train_dir)
valid_path = os.path.join(base_dir, valid_dir)
print(train_path)

dogs_breed\train


In [11]:
train_datagen = ImageDataGenerator(
 rescale=1./255,rotation_range=40,
 width_shift_range=0.2,
 height_shift_range=0.2,
 shear_range=0.2,
 zoom_range=0.2,
 horizontal_flip=True)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [12]:
train_gen = train_datagen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)
print(train_path)

Found 16469 images belonging to 120 classes.
dogs_breed\train


In [13]:
val_gen = val_datagen.flow_from_directory(
    valid_path,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 4111 images belonging to 120 classes.


# CNN

In [14]:
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import RMSprop

def create_model(use_dropout :bool = False):
  img_input = layers.Input(shape=(150, 150, 3))
  x = layers.Conv2D(16, 3, activation='relu')(img_input)
  x = layers.MaxPooling2D(2)(x)
  x = layers.Conv2D(32, 3, activation='relu')(x)
  x = layers.MaxPooling2D(2)(x)
  x = layers.Conv2D(64, 3, activation='relu')(x)
  x = layers.MaxPooling2D(2)(x)
  x = layers.Flatten()(x)
  x = layers.Dense(512, activation='relu')(x)

  if use_dropout:
      x = layers.Dropout(0.5)(x)

  # Create output layer
  output = layers.Dense(120, activation='softmax')(x) # species number


  if 'model' in locals():
      del model 

  model_intra = Model(img_input, output)

  model_intra.summary()

  model_intra.compile(
      loss='categorical_crossentropy',
      optimizer=RMSprop(lr=0.001),
      metrics=['acc']
  )
  return model_intra

model = create_model()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 17, 17, 64)        0   

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def train_model(
    train_img_nb :int, 
    valid_img_nb :int, 
    batch_size :int,
    epochs :int, 
    use_data_augmentation :bool = False
):
    # All images will be rescaled by 1./255
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    ) if use_data_augmentation else ImageDataGenerator(
        rescale=1./255
    )
    
    val_datagen = ImageDataGenerator(rescale=1./255)
    
    train_gen = train_datagen.flow_from_directory(
        train_path,
        target_size=(150, 150),
        batch_size=20,
        classes=os.listdir(train_path)
    )
    
    valid_generator = val_datagen.flow_from_directory(
        valid_path,
        target_size=(150, 150),
        batch_size=20,
        classes=os.listdir(valid_path)
    )

    history = model.fit(
        train_gen,
        steps_per_epoch=int(train_img_nb / batch_size),
        epochs=epochs,
        validation_data=valid_generator,
        validation_steps=int(valid_img_nb / batch_size),
        verbose=2
    )
    return history

history = train_model(
    16469, 
    4111, 
    batch_size=20,
    epochs=15
)



Found 16469 images belonging to 120 classes.
Found 4111 images belonging to 120 classes.
Epoch 1/15
823/823 - 127s - loss: 4.6013 - acc: 0.0287 - val_loss: 4.3931 - val_acc: 0.0507
Epoch 2/15
823/823 - 122s - loss: 4.0510 - acc: 0.0978 - val_loss: 4.0159 - val_acc: 0.0968
Epoch 3/15
823/823 - 121s - loss: 3.3628 - acc: 0.2141 - val_loss: 4.0485 - val_acc: 0.1112
Epoch 4/15
823/823 - 122s - loss: 2.1647 - acc: 0.4586 - val_loss: 5.0169 - val_acc: 0.1100
Epoch 5/15
823/823 - 124s - loss: 0.9175 - acc: 0.7528 - val_loss: 7.0064 - val_acc: 0.0890
Epoch 6/15
823/823 - 121s - loss: 0.3125 - acc: 0.9183 - val_loss: 10.0854 - val_acc: 0.0915
Epoch 7/15
823/823 - 124s - loss: 0.1826 - acc: 0.9531 - val_loss: 11.7869 - val_acc: 0.0907
Epoch 8/15
823/823 - 128s - loss: 0.1477 - acc: 0.9652 - val_loss: 12.9408 - val_acc: 0.0844
Epoch 9/15
823/823 - 129s - loss: 0.1332 - acc: 0.9700 - val_loss: 13.2082 - val_acc: 0.0905
Epoch 10/15
823/823 - 124s - loss: 0.1091 - acc: 0.9768 - val_loss: 13.7186 - v

# Data augmentation

In [ ]:
from tensorflow.keras import layers, Model

def model_create_augmentation():
    inputs = layers.Input(shape=(150, 150, 3))
    x = layers.Conv2D(16, 3, activation='relu')(inputs)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(32, 3, activation='relu')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(64, 3, activation='relu')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(120, activation='softmax')(x) # species number 

    m = Model(img_input, output)

    m.summary()

    m.compile(loss='categorical_crossentropy',optimizer=['rmsprop'],metrics=['acc'])

    return m

model = model_create_augmentation()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def train_model_augmentation():
    train_datagen = ImageDataGenerator(rescale=1./255)

    val_datagen = ImageDataGenerator(rescale=1./255)

    train_gen = train_datagen.flow_from_directory(
        train_path,
        target_size=(150, 150),
        batch_size=20,
        classes=os.listdir(train_path)
    )

    valid_gen = val_datagen.flow_from_directory(
        valid_path,
        target_size=(150, 150),
        batch_size=20,
        classes=os.listdir(valid_path)
    )

    algo = model.fit(
        train_gen,
        steps_per_epoch=int(16469 / 20),
        epochs=15,
        validation_data=valid_gen,
        validation_steps=int(4111 / 20)
    )
    return algo

algo = train_model_augmentation()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 17, 17, 64)        0   